In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy import stats
import warnings



from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error


In [3]:
iowa_file_path = '../input/home-data-for-ml-course/train.csv'
home_data = pd.read_csv(iowa_file_path)

test_file_path = '../input/home-data-for-ml-course/test.csv'
test_data = pd.read_csv(test_file_path)

In [4]:
num_features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', \
            'BedroomAbvGr', 'TotRmsAbvGrd', 'YearRemodAdd', 'OverallCond', \
            'OverallQual', 'BsmtFinSF1', 'GarageCars', 'HalfBath','GrLivArea', \
            'TotalBsmtSF', 'BsmtFinSF2', 'BsmtFullBath', 'Fireplaces', 'PoolArea']

cat_features = ['BldgType', 'CentralAir', 'BsmtQual', 'Street', 'MSZoning', \
                'BsmtExposure','FireplaceQu','PavedDrive','Neighborhood', \
               'GarageType', 'LotConfig']
all_features = num_features + cat_features
X = home_data.copy()[all_features]
y = home_data.SalePrice
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

test_X = test_data.copy()[all_features]

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

test_X = test_data.copy()[all_features]

numerical_cols = [cname for cname in train_X.columns if 
                                train_X[cname].dtype in ['int64', 'float64']]

categorical_cols = [cname for cname in train_X.columns if 
                                train_X[cname].nunique() < 30 and
                                train_X[cname].dtype == "object"]

numerical_transformer = SimpleImputer(strategy='constant')


categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])


In [5]:
model = XGBRegressor(n_estimators=2000,learning_rate=0.01)

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

clf.fit(train_X, train_y)

preds = clf.predict(val_X)
mae = mean_absolute_error(preds,val_y)
print("Mean Absolute Error:" , mae)

preds_test = clf.predict(test_X)

#output = pd.DataFrame({'Id': test_X.index,
 #                      'SalePrice': preds_test})
#output.to_csv('submission.csv', index=False)

sample_submission = pd.read_csv('../input/home-data-for-ml-course/sample_submission.csv')

output = pd.DataFrame({'Id': sample_submission.Id,
'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)




Mean Absolute Error: 14280.16584974315
